# 개요
* GPT활용교육을 들은 내용을 별도로 정리
  * 1일차 : 네이버API 활용 : GPT를 활용한 변수 분석 등
  * 2일차 : beautifulsoup활용한 뉴스 및 증권사리포트 크롤링
  * 3일차 : 뉴스 및 증권사리포트 데이터 전처리 
  * 4일차 : 감성점수 변환 및 시각화
  * 5일차 : 백테스팅 실습
* 자유과제 진행으로 일부만 구현한 내용 정리

# 제출내용 요약

* 과제명 : 연금저축계좌에 보유중인 ETF에 대한 상위 10개 종목의 감성분석 및 투자판단 보조

## 1. 주제 선정 및 문제 정의

- 연금저축계좌에 보유중인 ETF에 대한 상위 10개 종목의 감성분석 및 투자판단 보조

---

## 2. 적용 도구 및 기법

- ChatGPT 활용
- 프롬프트 예시는 다음 페이지 참조

---

## 3. 적용 시나리오 구성

- 연금저축계좌에 보유중인 ETF종목에 대한 구성종목 확인  
  (예: KODEX 코스닥 ETF는 알테오젠 등 보유)

- 각 ETF종목의 상위 10개 항목 추출 (상위 10개 항목에 대한 감성분석 예정)

  - ETF운용사 홈페이지에서 크롤링하여 상위 10개 항목 추출
  - 강의 내용처럼 GPT URL 분석 요청 후 변수 확인  
    (예: TIGER → ISIN / KODEX → 고유번호)

- 모든 상위 10개 항목을 모은 후, 증권사 리포트 확보 및 리포트별 감성분석 진행

- 긍정/부정 지표 추이를 ETF 비중(평가금액)에 반영하여 평가금액 변동 예상치 산출

---

### 예시 시나리오

1. KODEX 코스닥150의 상위 10개 종목(알테오젠, HLB 등)의 증권사 리포트를 크롤링  
2. 존재하는 리포트에 대해 감성분석을 진행하여 구성종목별 평가금액 조정을 진행  
   - 알테오젠 유지 판단 → 그대로  
   - HLB 상승 판단 → 평가금액에 조정배율 곱함  
3. 각 구성종목의 예상 평가금액을 합산하여 기존 평가금액합보다 크면 매수, 아니면 매도  
   - 예상 평가금액이 커지면 향후 가격 상승 가능성 → 매수 진행

---

## 4. 실행 결과 정리 (한계점 및 GPT 실행결과 요약)

- ETF 운용사에 따라 웹사이트에 데이터가 없거나, 채권형 ETF 등은 분석 적용이 어려움

---

## 5. 회고 및 개선 방향

- 원래는 네이버증권을 통해 단일 크롤링 코드를 구현하려 했으나,  
  로딩 방식 이슈로 어려움을 겪음 → 각 운용사별 URL 분석 방식으로 대체

- 향후 네이버증권 한 페이지에서 구성종목 및 여론(토론방) 정보까지  
  통합 분석할 수 있다면 효율적인 프로젝트 운영 가능

- 현재는 국내종목 위주로 구현했으나, 향후 해외종목에도 적용되도록 확장 예정

---

# 과제 구현내용 일부

## 종목선정 : 보유현황 데이터프레임으로 1차 정리

* ETF운용사에 따라 크롤링 방법이 다르므로, Crawl파라메터를 별도로 기입

In [23]:
from pykrx import stock

In [ ]:
import pandas as pd
from pykrx import stock
import re

etf_in_account = ['251350','360750','245340','319640','219480','214980','157450',
                  '195980','305080','182480','310970','148070','329200','069500','229200']

data = []
for ticker in etf_in_account:
    etf_name = stock.get_etf_ticker_name(ticker)
    etf_isin = stock.get_etf_isin(ticker)
    words = etf_name.split()

    # crawl_param 조건 지정
    if words[0] == "KODEX":
        crawl_param = None
    elif words[0] == "TIGER":
        crawl_param = etf_isin
    elif words[0] == "KIWOOM":
        crawl_param = ticker
    else:
        crawl_param = None  # 기타 매니저는 None 처리

    data.append({
        "Ticker": ticker,
        "ETF_Name": etf_name,
        "ETF_ISIN": etf_isin,
        "ETF_Manager": words[0],
        "ETF_crawl_param": crawl_param
        
    })

df_etf = pd.DataFrame(data)
df_etf

,Ticker,ETF_Name,ETF_ISIN,ETF_Manager,ETF_crawl_param
0,251350,KODEX 선진국MSCI World,KR7251350005,KODEX,None
1,360750,TIGER 미국S&P500,KR7360750004,TIGER,KR7360750004
2,245340,TIGER 미국다우존스30,KR7245340005,TIGER,KR7245340005
3,319640,TIGER 골드선물(H),KR7319640009,TIGER,KR7319640009
4,219480,KODEX 미국S&P500선물(H),KR7219480001,KODEX,None
5,214980,KODEX 단기채권PLUS,KR7214980005,KODEX,None
6,157450,TIGER 단기통안채,KR7157450008,TIGER,KR7157450008
7,195980,PLUS 신흥국MSCI(합성 H),KR7195980008,PLUS,None
8,305080,TIGER 미국채10년선물,KR7305080004,TIGER,KR7305080004
9,182480,TIGER 미국MSCI리츠(합성 H),KR7182480004,TIGER,KR7182480004


* KODEX는 고유코드를 사용하므로 업데이트

In [ ]:
kodex_code = {'251350':'2ETF68',
              '219480':'2ETF50',
              '214980':'2ETF48',
              '069500':'2ETF01',
              '229200':'2ETF54'}

# ETF_Manager가 KODEX인 경우에만 kodex_code를 참고하여 ETF_crawl_param을 업데이트
df_etf['ETF_crawl_param'] = df_etf.apply(
    lambda row: kodex_code.get(row['Ticker'], None) if row['ETF_Manager'] == 'KODEX' else row['ETF_crawl_param'],
    axis=1
)
df_etf

,Ticker,ETF_Name,ETF_ISIN,ETF_Manager,ETF_crawl_param
0,251350,KODEX 선진국MSCI World,KR7251350005,KODEX,2ETF68
1,360750,TIGER 미국S&P500,KR7360750004,TIGER,KR7360750004
2,245340,TIGER 미국다우존스30,KR7245340005,TIGER,KR7245340005
3,319640,TIGER 골드선물(H),KR7319640009,TIGER,KR7319640009
4,219480,KODEX 미국S&P500선물(H),KR7219480001,KODEX,2ETF50
5,214980,KODEX 단기채권PLUS,KR7214980005,KODEX,2ETF48
6,157450,TIGER 단기통안채,KR7157450008,TIGER,KR7157450008
7,195980,PLUS 신흥국MSCI(합성 H),KR7195980008,PLUS,None
8,305080,TIGER 미국채10년선물,KR7305080004,TIGER,KR7305080004
9,182480,TIGER 미국MSCI리츠(합성 H),KR7182480004,TIGER,KR7182480004


## 종목선정 : ETF 상위 10개 종목 가져오기(TIGER ETF)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_tiger_top10_constituents(ksd_fund_code: str) -> list:
    url = f"https://www.tigeretf.com/ko/product/search/detail/index.do?ksdFund={ksd_fund_code}"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)
    response.encoding = "utf-8"

    soup = BeautifulSoup(response.text, 'html.parser')

    top10_list_div = soup.find('div', class_='box-list-div')
    if not top10_list_div:
        print(f"[{ksd_fund_code}] TOP10 구성종목 박스를 찾을 수 없습니다.")
        return []

    top10_names = []
    for box in top10_list_div.find_all('div', class_='box-list'):
        name_tag = box.find('p')
        if name_tag:
            top10_names.append(name_tag.get_text(strip=True))

    return top10_names[:10]


In [ ]:
ksd_fund_code = "KR7360750004"
top10 = get_tiger_top10_constituents(ksd_fund_code)
print(top10)

['Microsoft Corp', 'NVIDIA Corp', 'Apple Inc', 'Amazon.com Inc', 'Meta Platforms Inc', 'Broadcom Inc', 'Alphabet Inc', 'Tesla Inc', 'Berkshire Hathaway Inc', 'Alphabet Inc']


## 종목선정 : ETF 상위 10개 종목 가져오기(KODEX ETF)

* 직접 크롤링은 불가하여 엑셀을 읽는 형태로 구현

In [8]:
import requests
import pandas as pd
from io import BytesIO

def get_kodex_top10_constituents(etf_code):
    # 다운로드 URL (모바일 버전)
    url = f"https://m.samsungfund.com/excel_pdf.do?fId={etf_code}&gijunYMD=20250528"

    # 헤더 설정 (브라우저처럼 위장)
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    # 요청 보내기
    response = requests.get(url, headers=headers)

    # 응답이 성공적인 경우 (200 OK)
    if response.status_code == 200:
        # 바이너리 응답을 메모리 버퍼로 감싸서 pandas로 읽기
        excel_data = BytesIO(response.content)
        df = pd.read_excel(excel_data, skiprows=2)

        # 수량이 양수이고 종목명이 '원화예금'이 아닌 행만 필터링
        df = df[(df['수량'] > 0) & (df['종목명'] != '원화예금')]

        # 비중(%) 기준 상위 10개 종목 정렬
        df_top10 = df.sort_values(by='비중(%)', ascending=False).head(10)
    else:
        print("요청 실패:", response.status_code)

    return df_top10

In [9]:
etf_code = '2ETF54'

df_kodex150 = get_kodex_top10_constituents(etf_code) # KODEX 코스닥150
df_kodex150

,번호,종목명,ISIN,종목코드,수량,비중(%),평가금액(원),현재가(원),등락(원)
1,2,알테오젠,KR7196170005,196170,207,0.120936,71208000,350000,-
2,3,HLB,KR7028300002,028300,591,0.052896,31145700,52900,-
3,4,펩트론,KR7087010005,087010,109,0.042578,25070000,230000,-
4,5,에코프로,KR7086520004,086520,501,0.032631,19213350,42100,-
5,6,에코프로비엠,KR7247540008,247540,230,0.031718,18676000,89000,-
6,7,리가켐바이오,KR7141080002,141080,131,0.025808,15196000,114700,-
7,8,파마리서치,KR7214450009,214450,32,0.024402,14368000,473500,-
8,9,에이비엘바이오,KR7298380007,298380,168,0.023568,13876800,80700,-
9,10,삼천당제약,KR7000250001,000250,71,0.018546,10919800,146000,-
10,11,레인보우로보틱스,KR7277810008,277810,40,0.018104,10660000,269000,-


## 감성분석 데이터 : 대상종목 증권사리포트 추출 (1종목 추출샘플)

In [2]:
# 작성된 코드
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', 50)

def crawl_naver_reports(write_from, write_to, ticker, page=1):
    url = "https://finance.naver.com/research/company_list.naver"
    params = {
        "keyword": "",
        "brokerCode": "",
        "searchType": "itemCode", #"writeDate",
        "writeFromDate": write_from,
        "writeToDate": write_to,
        "itemName": "",
        "itemCode": ticker,
        "page": page
    }

    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.select_one("table.type_1")
    rows = table.select("tr")[2:]  # 헤더와 빈줄 제외

    results = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) != 6:
            continue  # 유효하지 않은 데이터 줄 무시

        # 종목명 및 코드
        stock_tag = cols[0].find("a")
        stock_name = stock_tag.text.strip()
        stock_code = stock_tag["href"].split("code=")[-1]

        # 제목 및 상세 링크
        title_tag = cols[1].find("a")
        title = title_tag.text.strip()
        detail_link = "https://finance.naver.com/research/" + title_tag["href"]

        # 증권사
        broker = cols[2].text.strip()

        # 첨부파일 링크 (없을 수도 있음)
        file_tag = cols[3].find("a")
        pdf_link = file_tag["href"] if file_tag else ""

        # 작성일, 조회수
        date = cols[4].text.strip()
        views = cols[5].text.strip()

        results.append({
            "종목명": stock_name,
            "종목코드": stock_code,
            "제목": title,
            "리포트링크": detail_link,
            "PDF링크": pdf_link,
            "증권사": broker,
            "작성일": date,
            "조회수": views
        })

    return pd.DataFrame(results)

In [4]:
ticker = '087010'
temp_df = crawl_naver_reports("2025-05-01", "2025-05-31", ticker, page=1)
temp_df.head(5)

,종목명,종목코드,제목,리포트링크,PDF링크,증권사,작성일,조회수
0,펩트론,087010,부정적 시각에 매몰된 기업설명회의 가치,https://finance.naver.com/research/company_rea...,,신한투자증권,24.11.26,11581
1,펩트론,087010,릴리다. 실패는 없다,https://finance.naver.com/research/company_rea...,,신한투자증권,24.10.08,16780
2,펩트론,087010,GLP-1 전쟁의 캐스팅보트,https://finance.naver.com/research/company_rea...,https://ssl.pstatic.net/imgstock/upload/resear...,키움증권,23.07.18,19902
3,펩트론,087010,나를 다시 보게 될 거야,https://finance.naver.com/research/company_rea...,https://ssl.pstatic.net/imgstock/upload/resear...,키움증권,23.06.22,11820
4,펩트론,087010,독자적인 펩타이드 기반 플랫폼 기술을 보유..,https://finance.naver.com/research/company_rea...,https://ssl.pstatic.net/imgstock/upload/resear...,나이스디앤비,22.03.03,9006


## 감성분석 데이터 : 대상종목 증권사리포트 추출 (상위 10개)

In [15]:
list_ticker = df_kodex150['종목코드'].tolist()
list_ticker

['196170',
 '028300',
 '087010',
 '086520',
 '247540',
 '141080',
 '214450',
 '298380',
 '000250',
 '277810']

In [ ]:
ticker = '229200' # ETF티커
df_concat = pd.DataFrame() # 전체내용 취합용 빈 데이터프레임

for each_ticker in list_ticker:
    temp_df = crawl_naver_reports("2025-05-01", "2025-05-31", each_ticker, page=1)
    df_date_filtered = temp_df[temp_df['작성일'].str.contains('25.05')]
    if len(df_date_filtered)==0:
        pass
    else:
        df_date_filtered['ETF_name'] = stock.get_etf_ticker_name(ticker)
        df_date_filtered['ETF_ticker'] = ticker
        df_concat = pd.concat([df_concat, df_date_filtered])

In [32]:
df_concat

,종목명,종목코드,제목,리포트링크,PDF링크,증권사,작성일,조회수,ETF_name,ETF_ticker
0,알테오젠,196170,"IRA 약가 인하, 특허 이슈 종합 분석",https://finance.naver.com/research/company_rea...,,신한투자증권,25.05.27,4130,KODEX 코스닥150,229200
0,리가켐바이오,141080,돋보이는 ADC 기업,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,미래에셋증권,25.05.23,2921,KODEX 코스닥150,229200
1,리가켐바이오,141080,페이로드 강자로 도약중,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,유안타증권,25.05.22,2259,KODEX 코스닥150,229200
2,리가켐바이오,141080,LCB39: Cold to Hot,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,유진투자증권,25.05.07,2951,KODEX 코스닥150,229200
0,파마리서치,214450,브레이크가 고장나버린 리쥬란 트럭,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,미래에셋증권,25.05.21,3419,KODEX 코스닥150,229200
1,파마리서치,214450,"인바운드와 브랜드 파워, 환상의 선순환",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,교보증권,25.05.14,2176,KODEX 코스닥150,229200
2,파마리서치,214450,"멈추지 않는 실적 성장, 다만 주가는 트레이..",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,대신증권,25.05.14,1316,KODEX 코스닥150,229200
3,파마리서치,214450,"늦었다고 생각할 때가 진짜 늦었다, 그러니 ..",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,키움증권,25.05.14,1407,KODEX 코스닥150,229200
0,레인보우로보틱스,277810,"K-Humanoid, 제가 해보겠습니다",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,SK증권,25.05.30,1381,KODEX 코스닥150,229200


In [ ]:
df_concat['PDF링크'].iloc[1]

'https://stock.pstatic.net/stock-research/company/29/20250523_company_875723000.pdf'

## 감성분석 데이터 : 리포트 다운로드

In [ ]:
import os
import re
import requests

def sanitize_filename(text):
    # 파일명에 사용할 수 없는 문자 제거
    return re.sub(r'[\\/:*?"<>|]', '', text)

def download_pdfs(df, save_folder="pdf_reports"):
    os.makedirs(save_folder, exist_ok=True)

    for idx, row in df.iterrows():
        url = row['PDF링크']
        if not url:
            continue  # PDF 링크 없으면 스킵

        date = row['작성일'].replace('.', '-')  # 예: 23.03.27 → 23-03-27
        stock = sanitize_filename(row['종목명'])
        title = sanitize_filename(row['제목'])[:50]  # 너무 긴 제목 잘라내기

        filename = f"{date}_{stock}_{title}.pdf"
        filepath = os.path.join(save_folder, filename)

        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                with open(filepath, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"[✔] 저장됨: {filename}")
            else:
                print(f"[✖] 실패: {filename} - 상태코드 {response.status_code}")
        except Exception as e:
            print(f"[⚠️] 오류 발생 ({filename}): {e}")
        

In [ ]:
download_pdfs(df_concat)

[✔] 저장됨: 25-05-23_리가켐바이오_돋보이는 ADC 기업.pdf
[✔] 저장됨: 25-05-22_리가켐바이오_페이로드 강자로 도약중.pdf
[✔] 저장됨: 25-05-07_리가켐바이오_LCB39 Cold to Hot.pdf
[✔] 저장됨: 25-05-21_파마리서치_브레이크가 고장나버린 리쥬란 트럭.pdf
[✔] 저장됨: 25-05-14_파마리서치_인바운드와 브랜드 파워, 환상의 선순환.pdf
[✔] 저장됨: 25-05-14_파마리서치_멈추지 않는 실적 성장, 다만 주가는 트레이...pdf
[✔] 저장됨: 25-05-14_파마리서치_늦었다고 생각할 때가 진짜 늦었다, 그러니 ...pdf
[✔] 저장됨: 25-05-30_레인보우로보틱스_K-Humanoid, 제가 해보겠습니다.pdf


## 감성분석 데이터 : 조사한 내용으로 OPENAI API로 리포트 요약 

In [ ]:
# pandas apply에 활용할 OpenAI로 요약해주는 함수 작성

from PyPDF2 import PdfReader
from openai import OpenAI
import json

# OpenAI API key
with open("keys.json", "r") as f:
    data = json.load(f)

open_ai_key = data.get("open_ai_key")

def generate_summary(row):
    # pdf 파일을 가져와서 이를 PyPDF2에 넣어서 text를 추출
    date = row['작성일'].replace('.', '-')  # 예: 23.03.27 → 23-03-27
    stock = sanitize_filename(row['종목명'])
    title = sanitize_filename(row['제목'])[:50]  # 너무 긴 제목 잘라내기

    file_name = f"{date}_{stock}_{title}.pdf"

    try:
        reader = PdfReader(f"pdf_reports/{file_name}")
        text = reader.pages[0].extract_text()
    except:
        return None

    prompt = f"""
    아래의 증권사 리포트 내용을 바탕으로 간단하고 명확한 요약을 생성해주세요.
    요약은 다음과 같은 요건을 충족해야 합니다:
    1. 증권사 리포트에 담긴 주요 정보와 메시지를 간결하게 포함할 것.
    2. 긍정적, 부정적 또는 중립적인 뉘앙스를 명확히 드러낼 것. 감정이 모호하지 않도록 작성해주세요.
    3. 독자가 감성 분석을 하기 쉽도록 명료한 문장으로 작성할 것.

    내용: {text}

    아래에 간결한 요약을 작성해주세요:
    """

    client = OpenAI(api_key=open_ai_key)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user","content": prompt}]
    )

    print(completion.choices[0].message.content)

    return completion.choices[0].message.content

In [ ]:
df_concat['generated_summary'] = df_concat.apply(generate_summary, axis=1)

리가켐바이오는 KOSDAQ에서 주가 상승세를 보이며 2023년 24.5에서 25.5로 오름세를 기록했습니다. 이러한 긍정적인 변화는 투자자들에게 희망적인 신호를 주고 있으며, 앞으로의 성장을 기대할 수 있는 상황입니다.
**요약:**  
금융투자분석사는 리가켐바이오(141080)에 대해 긍정적으로 평가하며, "BUY" 의견과 함께 목표주가를 15만원으로 설정했습니다. 현재주가는 10만9300원으로, 상승 여력은 37%로 판단합니다. 리가켐바이오는 pPBD 및 LCB39로 강력한 페이로드 기술을 보유하고 있으며, 안전성과 성과가 개선될 것으로 기대됩니다. 향후 AVANZAR 임상 결과에 따라 LCB84의 가치 또한 상승할 것으로 전망됩니다. 향후 실적 예측에 따르면, 2024년에는 영업적자가 줄어들고 2025년에는 지배순이익이 흑자로 전환될 것으로 예상됩니다. 전체적으로 이 회사는 강력한 성장 가능성을 지니고 있습니다.
리가켐바이오의 LCB39에 대한 최근 리포트는 이 후보 물질이 기존 치료법에 비해 혁신적인 가능성을 보여주고 있음을 강조하고 있습니다. 연구 결과에 따르면 LCB39는 종양에 대한 효과적인 반응을 나타내며, 앞으로의 임상 결과에 대한 기대가 큽니다. 이러한 긍정적인 발전에도 불구하고, 추가 연구와 임상이 필요하다는 점을 주의 깊게 언급하고 있습니다. 전반적으로 본 리포트는 리가켐바이오의 가능성에 대한 긍정적인 전망을 제시하고 있습니다.
파마리서치(KOSDAQ)의 주가는 최근 24.5에서 25.5 사이에서 변동하고 있으며, 전반적으로 상승세를 보이고 있습니다. 이러한 경향은 긍정적이며, 투자자들에게 희망적인 신호로 해석될 수 있습니다.
**파마리서치 분석 요약**

파마리서치의 투자 의견은 **BUY**로 유지되며 목표 주가는 **450,000원**으로 상향 조정되었습니다. 1분기 실적은 매출 1,169억원(전년 대비 56.5% 증가)과 영업이익 447억원(전년 대비 67.6% 증가)으로 컨센서스를 초과 달성했습니다. 의료기기 부문이 81% 성장하

In [ ]:
df_concat

,종목명,종목코드,제목,리포트링크,PDF링크,증권사,작성일,조회수,ETF_name,ETF_ticker,generated_summary
0,알테오젠,196170,"IRA 약가 인하, 특허 이슈 종합 분석",https://finance.naver.com/research/company_rea...,,신한투자증권,25.05.27,3457,KODEX 코스닥150,229200,None
0,리가켐바이오,141080,돋보이는 ADC 기업,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,미래에셋증권,25.05.23,2798,KODEX 코스닥150,229200,리가켐바이오는 KOSDAQ에서 주가 상승세를 보이며 2023년 24.5에서 25.5...
1,리가켐바이오,141080,페이로드 강자로 도약중,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,유안타증권,25.05.22,2205,KODEX 코스닥150,229200,**요약:** \n금융투자분석사는 리가켐바이오(141080)에 대해 긍정적으로 평...
2,리가켐바이오,141080,LCB39: Cold to Hot,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,유진투자증권,25.05.07,2917,KODEX 코스닥150,229200,리가켐바이오의 LCB39에 대한 최근 리포트는 이 후보 물질이 기존 치료법에 비해 ...
0,파마리서치,214450,브레이크가 고장나버린 리쥬란 트럭,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,미래에셋증권,25.05.21,2990,KODEX 코스닥150,229200,파마리서치(KOSDAQ)의 주가는 최근 24.5에서 25.5 사이에서 변동하고 있으...
1,파마리서치,214450,"인바운드와 브랜드 파워, 환상의 선순환",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,교보증권,25.05.14,2073,KODEX 코스닥150,229200,**파마리서치 분석 요약**\n\n파마리서치의 투자 의견은 **BUY**로 유지되며...
2,파마리서치,214450,"멈추지 않는 실적 성장, 다만 주가는 트레이..",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,대신증권,25.05.14,1238,KODEX 코스닥150,229200,파마리서치의 1분기 실적이 전년 대비 56% 증가하며 고성장을 이어갔습니다. 매출 ...
3,파마리서치,214450,"늦었다고 생각할 때가 진짜 늦었다, 그러니 ..",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,키움증권,25.05.14,1327,KODEX 코스닥150,229200,"**요약:**\n\n파마리서치의 2025년 1분기 실적이 양호하며, 매출액 1,16..."
0,레인보우로보틱스,277810,"K-Humanoid, 제가 해보겠습니다",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,SK증권,25.05.30,936,KODEX 코스닥150,229200,레인보우로보틱스(KQ/277810)는 인간형 이족보행 로봇 'HUBO' 개발을 바탕...


## 감성분석 진행 : Openai API로 감성분석 진행

In [ ]:
# pandas apply에 활용할 OpenAI로 요약해주는 함수 작성

from openai import OpenAI
import json

# OpenAI API key
with open("keys.json", "r") as f:
    data = json.load(f)

open_ai_key = data.get("open_ai_key")

# 함수 작성
def get_sentiment_score(row):
    prompt = f"""
            당신은 주식과 관련된 뉴스 또는 증권사 리포트에서 감성 점수를 추출하는 전문가입니다.
            주어진 텍스트를 분석하여 감성 점수를 다음의 기준에 따라 평가하세요.

            - 긍정 (Positive): 해당 텍스트가 주식의 긍정적인 영향을 암시하는 경우
            - 중립 (Neutral): 해당 텍스트가 중립적이거나 영향을 판단하기 어려운 경우
            - 부정 (Negative): 해당 텍스트가 주식에 부정적인 영향을 암시하는 경우
            각 긍정, 중립, 부정 점수의 합이 1이 되게 해주세요.

            다음은 입력 데이터와 기대하는 출력 데이터의 예시입니다.
            예시 1:
            입력: 해당 회사의 매출이 25% 증가하며 시장 기대치를 초과했습니다.
            출력: 긍정 0.83 중립 0.13 부정 0.04
            예시 2:
            입력: 글로벌 경제 둔화로 인해 매출 감소가 예상됩니다.
            출력: 긍정 0.03 중립 0.23 부정 0.74
            예시 3:
            입력: 회사는 안정적인 성장을 유지하고 있으며, 큰 변화가 없습니다.
            출력: 긍정 0.13 중립 0.83 부정 0.04

            아래에 주어진 텍스트를 분석하고, 감성 점수를 출력 형식에 맞춰 응답하세요.

            {row['generated_summary']}
            """


    client = OpenAI(api_key =open_ai_key
                    )
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user","content": prompt}]
    )

    # print(completion.choices[0].message.content)

    results = completion.choices[0].message.content
    positive = float(results.split("긍정 ")[1].split(' ')[0].strip())
    neutral = float(results.split("중립 ")[1].split(' ')[0].strip())
    negative = float(results.split("부정 ")[1].split(' ')[0].strip())
    # print(f"{positive} {neutral} {negative}")

    return positive, neutral, negative

In [ ]:
df_concat[['positive_score', 'neutral_score', 'negative_score']] = df_concat.apply(lambda row: pd.Series(get_sentiment_score(row)), axis=1)

In [ ]:
df_concat

,종목명,종목코드,제목,리포트링크,PDF링크,증권사,작성일,조회수,ETF_name,ETF_ticker,generated_summary,positive_score,neutral_score,negative_score
0,알테오젠,196170,"IRA 약가 인하, 특허 이슈 종합 분석",https://finance.naver.com/research/company_rea...,,신한투자증권,25.05.27,3457,KODEX 코스닥150,229200,None,0.00,0.00,0.00
0,리가켐바이오,141080,돋보이는 ADC 기업,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,미래에셋증권,25.05.23,2798,KODEX 코스닥150,229200,리가켐바이오는 KOSDAQ에서 주가 상승세를 보이며 2023년 24.5에서 25.5...,0.75,0.15,0.10
1,리가켐바이오,141080,페이로드 강자로 도약중,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,유안타증권,25.05.22,2205,KODEX 코스닥150,229200,**요약:** \n금융투자분석사는 리가켐바이오(141080)에 대해 긍정적으로 평...,0.85,0.10,0.05
2,리가켐바이오,141080,LCB39: Cold to Hot,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,유진투자증권,25.05.07,2917,KODEX 코스닥150,229200,리가켐바이오의 LCB39에 대한 최근 리포트는 이 후보 물질이 기존 치료법에 비해 ...,0.75,0.20,0.05
0,파마리서치,214450,브레이크가 고장나버린 리쥬란 트럭,https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,미래에셋증권,25.05.21,2990,KODEX 코스닥150,229200,파마리서치(KOSDAQ)의 주가는 최근 24.5에서 25.5 사이에서 변동하고 있으...,0.75,0.20,0.05
1,파마리서치,214450,"인바운드와 브랜드 파워, 환상의 선순환",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,교보증권,25.05.14,2073,KODEX 코스닥150,229200,**파마리서치 분석 요약**\n\n파마리서치의 투자 의견은 **BUY**로 유지되며...,0.88,0.09,0.03
2,파마리서치,214450,"멈추지 않는 실적 성장, 다만 주가는 트레이..",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,대신증권,25.05.14,1238,KODEX 코스닥150,229200,파마리서치의 1분기 실적이 전년 대비 56% 증가하며 고성장을 이어갔습니다. 매출 ...,0.75,0.15,0.10
3,파마리서치,214450,"늦었다고 생각할 때가 진짜 늦었다, 그러니 ..",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,키움증권,25.05.14,1327,KODEX 코스닥150,229200,"**요약:**\n\n파마리서치의 2025년 1분기 실적이 양호하며, 매출액 1,16...",0.90,0.07,0.03
0,레인보우로보틱스,277810,"K-Humanoid, 제가 해보겠습니다",https://finance.naver.com/research/company_rea...,https://stock.pstatic.net/stock-research/compa...,SK증권,25.05.30,936,KODEX 코스닥150,229200,레인보우로보틱스(KQ/277810)는 인간형 이족보행 로봇 'HUBO' 개발을 바탕...,0.55,0.35,0.10


## 감성분석 진행 : GPT제안에 따른 ETF비중별 감성분석 적용

* GPT에게 퀀트가 감성분석 적용하는 일반적인 경우를 요청하는 프롬프트
  * 아래 표는 구성종목의 감성분석을 진행한 결과야. 감성분석 결과를 비중(금액기준)을 기준으로 반영해 매도/매수할지 판단에 보조를 하고싶을때, 퀀트업무를 하는사람은 보통 어떤 식으로 구성할까?


* GPT답변


아래는 이미지 내용을 마크다운으로 변환한 결과입니다:


🎯 **가장 많이 쓰이는 3가지 방식**

---

📌 A. 선형 가중 조정 (Linear Reweighting)

```python
adjusted_weight = 기존비중 × (1 + 감성점수)

* 감성 점수에 따라 비중을 직접적으로 조정
* 비중 총합이 1이 되도록 정규화 필요
```
---

📌 B. 분류 기반 조정 (Categorical)

* 점수에 따라 3\~5개 등급 구분 후, 비중을 고정 비율로 재분배

| 감성 점수       | 조정 방식       |
| ----------- | ----------- |
| > 0.5       | 기존 비중의 1.2배 |
| -0.5 \~ 0.5 | 그대로         |
| < -0.5      | 기존 비중의 0.8배 |

---

📌 C. Z-score 기반 이상치 판단

* 전체 감성 점수의 분포를 표준화한 후 ±1σ 이상인 종목만 비중 조정


In [ ]:
import pandas as pd

# 1. 감성 점수 계산
df_concat["sentiment_score"] = df_concat["positive_score"] - df_concat["negative_score"]

# 2. 종목코드를 기준으로 merge
df_merged = pd.merge(df_kodex150, df_concat, how="left", on="종목코드")

# 3. 분류 기반 조정 비율 계산 함수
def get_weight_multiplier(score):
    if score > 0.5:
        return 1.2
    elif score < -0.5:
        return 0.8
    else:
        return 1.0

# 4. 적용
df_merged["비중(%)"] = df_merged["비중(%)"].astype(float)
df_merged["조정배율"] = df_merged["sentiment_score"].apply(get_weight_multiplier)
df_merged["조정비중(%)"] = df_merged["비중(%)"] * df_merged["조정배율"]

# 5. 비중 변화 및 매수/매도 판단
df_merged["비중차이(%)"] = df_merged["조정비중(%)"] - df_merged["비중(%)"]
df_merged["판단"] = df_merged["조정배율"].map({1.2: "매수", 1.0: "유지", 0.8: "매도"})

# 결과 확인
df_merged

,번호,종목명_x,ISIN,종목코드,수량,비중(%),평가금액(원),현재가(원),등락(원),종목명_y,...,ETF_ticker,generated_summary,positive_score,neutral_score,negative_score,sentiment_score,조정배율,조정비중(%),비중차이(%),판단
0,2,알테오젠,KR7196170005,196170,207,0.120936,71208000,330000,-4500,알테오젠,...,229200,None,0.00,0.00,0.00,0.00,1.0,0.120936,0.000000,유지
1,3,HLB,KR7028300002,028300,591,0.052896,31145700,54600,-100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.052896,0.000000,유지
2,4,펩트론,KR7087010005,087010,109,0.042578,25070000,232500,7500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.042578,0.000000,유지
3,5,에코프로,KR7086520004,086520,501,0.032631,19213350,42800,-250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.032631,0.000000,유지
4,6,에코프로비엠,KR7247540008,247540,230,0.031718,18676000,89700,-300,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.031718,0.000000,유지
5,7,리가켐바이오,KR7141080002,141080,131,0.025808,15196000,114900,400,리가켐바이오,...,229200,리가켐바이오는 KOSDAQ에서 주가 상승세를 보이며 2023년 24.5에서 25.5...,0.75,0.15,0.10,0.65,1.2,0.030970,0.005162,매수
6,7,리가켐바이오,KR7141080002,141080,131,0.025808,15196000,114900,400,리가켐바이오,...,229200,**요약:** \n금융투자분석사는 리가켐바이오(141080)에 대해 긍정적으로 평...,0.85,0.10,0.05,0.80,1.2,0.030970,0.005162,매수
7,7,리가켐바이오,KR7141080002,141080,131,0.025808,15196000,114900,400,리가켐바이오,...,229200,리가켐바이오의 LCB39에 대한 최근 리포트는 이 후보 물질이 기존 치료법에 비해 ...,0.75,0.20,0.05,0.70,1.2,0.030970,0.005162,매수
8,8,파마리서치,KR7214450009,214450,32,0.024402,14368000,451000,8000,파마리서치,...,229200,파마리서치(KOSDAQ)의 주가는 최근 24.5에서 25.5 사이에서 변동하고 있으...,0.75,0.20,0.05,0.70,1.2,0.029282,0.004880,매수
9,8,파마리서치,KR7214450009,214450,32,0.024402,14368000,451000,8000,파마리서치,...,229200,**파마리서치 분석 요약**\n\n파마리서치의 투자 의견은 **BUY**로 유지되며...,0.88,0.09,0.03,0.85,1.2,0.029282,0.004880,매수


## 감성분석 적용 : 예상 평가금액을 조정/합산하여 ETF예상가치 산출

* 위에서 계산한 조정비율을 종목별 평가금액에 반영, 합산하여 미래 ETF가치를 산출
  * 기존보다 총합산액(ETF내 모든 종목의 평가금액 합)이 커지면 가치상승으로 예상하여 매수
  * 기존보다 총합산액(ETF내 모든 종목의 평가금액 합)이 작아지면 가치하락으로 예상하여 매도
* 아래 케이스로는 상승으로 예상되어 매수 진행함

In [ ]:
import pandas as pd

# 평가금액 문자열이라면 숫자로 변환
df_merged["평가금액(원)"] = df_merged["평가금액(원)"].astype(int)

# 조정된 평가금액 계산
df_merged["조정평가금액(원)"] = df_merged["평가금액(원)"] * df_merged["조정배율"]

# ETF 총액 계산
total_eval_before = df_merged["평가금액(원)"].sum()
total_eval_after = df_merged["조정평가금액(원)"].sum()
change_ratio = (total_eval_after / total_eval_before - 1) * 100

# 결과 출력
print(f"✅ 원래 ETF 총 평가금액: {total_eval_before:,.0f} 원")
print(f"✅ 조정된 ETF 총 평가금액: {total_eval_after:,.0f} 원")
print(f"📊 변화율: {change_ratio:.2f}%")


✅ 원래 ETF 총 평가금액: 303,829,650 원
✅ 조정된 ETF 총 평가금액: 324,441,650 원
📊 변화율: 6.78%
